<span style="font-family:Papyrus; font-size:3em;">Cross Validation</span>

This notebook describes how to do cross validation with simulation models.

# Programming Preliminaries

In [182]:
!pip install -q tellurium
!pip install -q SBstoat

In [183]:
%matplotlib inline
import tellurium as te
import numpy as np
import math
import random 
import matplotlib.pyplot as plt
import urllib
from SBstoat import ModelFitter, NamedTimeseries

In [184]:
def getSharedCodes(moduleName):
  """
  Obtains common codes from the github repository.

  Parameters
  ----------
  moduleName: str
      name of the python module in the src directory
  """
  url = "https://github.com/vporubsky/network-modeling-summer-school/raw/main/src/%s.py" % moduleName
  local_python = "python.py"
  _, _ = urllib.request.urlretrieve(url=url, filename=local_python)
  with open(local_python, "r") as fd:
    codeStr = "".join(fd.readlines())
  print(codeStr)
  exec(codeStr, globals())

# Acquire codes
getSharedCodes("util")

# TESTS
assert(isinstance(LINEAR_PATHWAY_DF, pd.DataFrame))

import pandas as pd
import urllib.request

# Linear pathway data
BASE_URL = "https://github.com/vporubsky/network-modeling-summer-school/raw/main/"
BASE_DATA_URL = "%sdata/" % BASE_URL
BASE_MODULE_URL = "%ssrc/" % BASE_URL
BASE_MODEL_URL = "%smodels/" % BASE_URL
LOCAL_FILE = "local_file.txt"


def getData(csvFilename):
    """
    Creates a dataframe from a CSV structured URL file.

    Parameters
    ----------
    csvFilename: str
        Name of the CSV file (w/o ".csv" extension)

    Returns
    -------
    pd.DataFrame
    """
    url = "%s%s.csv" % (BASE_DATA_URL, csvFilename)
    filename, _ = urllib.request.urlretrieve(url, filename=LOCAL_FILE)
    return pd.read_csv(LOCAL_FILE)

def getModule(moduleName):
    """
    Obtains common codes from the github repository.
  
    Parameters
    ----------
    moduleName: str
        name of the python module in the src directory
    """
    url = "%s%s.py" % (BASE_MODULE_URL, moduleName)
    _, _ = urllib.request.urlretrieve(url, fil

In [185]:
def findCloseMatchingValues(longArr, shortArr):
    """
    Finds the indices in longArr that are closest to the values in shortArr.

    Parameters
    ----------
    longArr: np.array
    shortArr: np.arry

    Returns
    -------
    array-int
    """
    indices = []
    for val in shortArr:
        distances = (longArr - val)**2
        minDistance = np.min(distances)
        distancesLst = list(distances)
        idx = distancesLst.index(minDistance)
        indices.append(idx)
    return np.array(indices)

# TESTS
longArr = np.array(range(10))
shortArr = np.array([2.1, 2.9, 4.3])
indexArr = findCloseMatchingValues(longArr, shortArr)
expectedArr = [2, 3, 4]
assert(all([v1 == v2 for v1, v2 in zip(indexArr, expectedArr)]))

In [186]:
# Experimental conditions
END_TIME = max(LINEAR_PATHWAY_ARR[:, 0])
NUM_POINT = 15
NOISE_STD = 1.0
NUM_FOLD = 3

# Model

In [187]:
print(LINEAR_PATHWAY_MODEL)

R1:  S1 -> S2; k1*S1  
R2: S2 -> S3; k2*S2
R3: S3 -> S4; k3*S3
R4: S4 -> S5; k4*S4

S1 = 10

// Parameters
k1 = 0; # Nominal value of parameter
k2 = 0; # Nominal value of parameter
k3 = 0; # Nominal value of parameter
k4 = 0; # Nominal value of parameter



# Helper Functions

The following functions provide capabilities used in this notebook.

In [240]:
def runSimulation(simTime=END_TIME, numPoint=NUM_POINT, roadRunner=None,
                  parameterDct=None, model=LINEAR_PATHWAY_MODEL):
    """
    Runs the simulation model for the parameters.
   
    Parameters
    ----------
    endTime: float
        End time for the simulation
    numPoints: int
        Number of points in the simulation
    roadRunner: ExtendedRoadRunner
    parameters: list-str
        
    Returns
    -------
    NamedArray
        results of simulation
    """
    if roadRunner is None:
        roadRunner = te.loada(model)
    else:
        roadRunner.reset()
    if parameterDct is not None:
        # Set the simulation constants for all parameters
        for name in parameterDct.keys():
            roadRunner[name] = parameterDct[name]
    return roadRunner.simulate (0, simTime, numPoint)

# TESTS
numPoint = int(10*END_TIME)
fittedData = runSimulation(parameterDct={"k1": 0.1}, numPoint=numPoint)
numCol = np.shape(fittedData)[1]
assert(np.size(fittedData) == numPoint*numCol)
assert(fittedData[0, 1] == 10)

In [189]:
def plotFittedAndObserved(fittedData=FITTED_DATA, 
                          observedData=LINEAR_PATHWAY_ARR,
                          indices=None, title="", isPlot=True):
    """
    Plots the fitted and observed data

    Parameters
    ----------
    fittedData: np.array(N, M)
    observedData: np.array(N, M)
    indices: np.array(K), K <= N
    title: str
    isPlot: bool
    """
    # Plot the simulation data.
    numRow, numCol = np.shape(fittedData)
    if indices is None:
        indices = [v for v in range(numRow)]            
    fig, ax = plt.subplots(1)
    for col in range(1, numCol):
        ax.plot(fittedData[indices, 0], fittedData[indices, col])
        ax.scatter(fittedData[indices, 0], observedData[indices, col])
    ax.set_xlabel("Time")
    ax.set_ylabel("Concentration")
    ax.set_title(title)
    ax.legend(['A', 'B', 'C'])
    if isPlot:
        plt.show()
    else:
        fig.clear()
    
# Tests
indices = [i for i in range(NUM_POINT) if i % 3 == 0] # every third point
plotFittedAndObserved(indices=indices, isPlot=False)

<Figure size 432x288 with 0 Axes>

# Cross Validation Algorithm

Cross validation divides the data into training and test sets.
The training sets are used to estimate parameters.
The test sets are used to evaluate the quality of the
parameter fits.
That is, for each set of test indices, we have a companion set of training indices. The training indices select a subset of the observational data that are used to estimate parameter values.

Below is pseudo code that calculates the $R^2$ value for each fold.

    def crossValidate(model, observedData, numFold):
        folds = generateFolds(observedData, numFold)
        foldQualities = []
        for fold folds:
            foldQuality = evaluateFold(model, fold)
            foldQualities.append(foldQuality)
        return foldQualities

# Generating Folds

The first step is to generate the folds.
A fold is represented by a tuple.
The first element is the training data;
the second element is the test data.
The collection of these tuples is itsself an array.
``generateFolds`` such an array.



In [190]:
def generateFolds(observedData, numFold):
    """
    Generates indices of training and test data
    by alternating between folds
    
    Parameters:
    ----------
    observedData: np.array(N, M) or NamedTimeseries
    numFold: int
        number of pairs of testIndices and trainIndices
    
    Returns:
    --------
    array-tuple(array, array)
    """
    if isinstance(observedData, NamedTimeseries):
        df = observedData.to_dataframe()
        df = df.reset_index()
        observedData = df.to_numpy()
    result = []
    numPoint, numCol = np.shape(observedData)
    indices = range(numPoint)
    for remainder in range(numFold):
        testIndices = np.array([n for n in indices if n % numFold == remainder])
        testData = observedData[testIndices, :]
        trainIndices = np.array(list(set(indices).difference(testIndices)))
        trainData = observedData[trainIndices, :]
        entry = (trainData, testData)
        result.append(entry)
    return result

# TESTS
numFold = 3
folds = generateFolds(OBSERVED_DATA, numFold)
assert(len(folds) == numFold)
fold = folds[0]
assert(len(fold) == 2)
trainData = fold[0]
testData = fold[1]
assert(len(OBSERVED_DATA) == (len(trainData) + len(testData)))
#
observedData = NamedTimeseries(array=OBSERVED_DATA, colnames=FITTED_DATA.colnames)
fold2s = generateFolds(observedData, numFold)
assert(len(folds) == len(fold2s))

In [191]:
folds

[( [[ 3.33339,  4.18629,  1.03926,  1.3939],
    [ 3.84392,  5.09397,  1.92858,       0],
    [ 9.33233,  1.72711,  1.45016, 2.91192],
    [  11.797,   2.7348, 0.574518, 2.80401],
    [ 15.4377,  2.35749,  3.36049, 2.83769],
    [ 15.3839,  0.22828,  1.61582, 1.93528],
    [ 18.1407,        0, 0.704198, 4.02233],
    [ 24.0794,        0, 0.194332, 4.07845],
    [ 27.9118, 0.705078,  1.23688, 5.31472],
    [ 29.5184,        0,        0, 6.33264]],
   [[       0,  6.28784, 0.571879,  1.23834],
    [ 5.94041,  3.03971,  1.68857, 0.159584],
    [ 13.3165, 0.468339,        0,  3.21624],
    [ 20.4788, 0.417834, 0.674715,  4.78747],
    [ 26.0478,  0.53748,        0,  4.92594]]),
 ( [[       0,  6.28784, 0.571879,  1.23834],
    [ 3.84392,  5.09397,  1.92858,        0],
    [ 5.94041,  3.03971,  1.68857, 0.159584],
    [  11.797,   2.7348, 0.574518,  2.80401],
    [ 13.3165, 0.468339,        0,  3.21624],
    [ 15.3839,  0.22828,  1.61582,  1.93528],
    [ 20.4788, 0.417834, 0.674715,  4.787

In [192]:
trainData

 [[ 3.33339,  4.18629,  1.03926,  1.3939],
  [ 3.84392,  5.09397,  1.92858,       0],
  [ 9.33233,  1.72711,  1.45016, 2.91192],
  [  11.797,   2.7348, 0.574518, 2.80401],
  [ 15.4377,  2.35749,  3.36049, 2.83769],
  [ 15.3839,  0.22828,  1.61582, 1.93528],
  [ 18.1407,        0, 0.704198, 4.02233],
  [ 24.0794,        0, 0.194332, 4.07845],
  [ 27.9118, 0.705078,  1.23688, 5.31472],
  [ 29.5184,        0,        0, 6.33264]]

In [193]:
testData

 [[       0,  6.28784, 0.571879,  1.23834],
  [ 5.94041,  3.03971,  1.68857, 0.159584],
  [ 13.3165, 0.468339,        0,  3.21624],
  [ 20.4788, 0.417834, 0.674715,  4.78747],
  [ 26.0478,  0.53748,        0,  4.92594]]

# Evaluating Folds

In [242]:
def evaluateFold(model, colnames, parametersToFit, fold,
                 **fittingArgs):
    """
    Calculates the R-squared value for the fit of the predicted test data,
    whose parameters are estimated from the training data, with the observed
    tests data.

    Parameters
    ----------
    model: antimony/ExtendedRoadRunner
    colnames: list-str
        names of data columns
    fold: tuple(np.array, np.array)
        train data, test data
    fittingArgs: dict
        optional arguments for ModelFitter

    Returns
    -------
    float: R squared
    """
    # Estimate the parameters
    observedTS = NamedTimeseries(colnames=colnames, array=fold[0])
    fitter = ModelFitter(modelSpecification=model,
                                    parametersToFit=parametersToFit,
                                    observedData=observedTS,
                                    **fittingArgs,
                                    )
    fitter.fitModel()
    parameterDct = dict(fitter.params.valuesdict())
    # Obtain the fitted values for the estimated parameters
    if "endTime" in fittingArgs:
        endTime = fittingArgs["endTime"]
    else:
        endTime = END_TIME
    numPoint = int(10*endTime)
    fittedData = runSimulation(simTime=endTime, numPoint=numPoint,
                                parameterDct=parameterDct,
                                roadRunner=model)
    # Find the time indices that correspond to the test data
    testData = fold[1]
    testTimes = testData[:, 0]
    fittedTimes = fittedData[:, 0]
    indices = findCloseMatchingValues(fittedTimes, testTimes)
    # Calculate residuals for the corresponding times
    indexArr = np.array(indices)
    fittedTestData = fittedData[indexArr, 1:]
    flatFittedTestData = fittedTestData.flatten()
    flatTestData = (testData[:, 1:]).flatten()
    residualsArr = flatTestData - flatFittedTestData
    rsq = 1 - np.var(residualsArr)/np.var(flatTestData)
    #
    return rsq

# TESTS
model = te.loada(LINEAR_PATHWAY_MODEL)
colnames = list(LINEAR_PATHWAY_DF.columns)
parametersToFit = [
                SBstoat.Parameter("k1", lower=0, value=0, upper=10),
                SBstoat.Parameter("k2", lower=0, value=0, upper=10),
                SBstoat.Parameter("k3", lower=0, value=0, upper=10),
                SBstoat.Parameter("k4", lower=0, value=0, upper=10),
    ]
folds = generateFolds(LINEAR_PATHWAY_ARR, NUM_FOLD)
rsq = evaluateFold(model, colnames, parametersToFit, folds[0],
                   fitterMethods=["differential_evolution"])
assert(rsq > 0.85)

# Complete Workflow

In [222]:
def crossValidate(model, observedData, parametersToFit, colnames, numFold,
                  **fitterArgs):
    """
    Performs cross validation on the model.

    Parameters
    ----------
    model: ExtendedRoadrunner
    observedData: NamedTimeseries
    parametersToFit: list-SBstoat.Parameter
    colnames: list-str
    numFold: int

    Results
    -------
    list-float: R-squared value for each fold
    """
    folds = generateFolds(observedData, numFold)
    foldQualities = []
    for fold in folds:
        foldQuality = evaluateFold(model, colnames, parametersToFit, fold,
                                   **fitterArgs)
        foldQualities.append(foldQuality)
    return foldQualities

# TESTS
model = te.loada(LINEAR_PATHWAY_MODEL)
colnames = list(LINEAR_PATHWAY_DF.columns)
parametersToFit = [
                SBstoat.Parameter("k1", lower=0, value=0, upper=10),
                SBstoat.Parameter("k2", lower=0, value=0, upper=10),
                SBstoat.Parameter("k3", lower=0, value=0, upper=10),
                SBstoat.Parameter("k4", lower=0, value=0, upper=10),
    ]
observedData = NamedTimeseries(dataframe=LINEAR_PATHWAY_DF)
qualities = crossValidate(model, observedData, parametersToFit,
                          colnames, NUM_FOLD,
                          fitterMethods=["differential_evolution"])
trues = [q > 0.85 for q in qualities]
assert(all(trues))

In [211]:
qualities

[0.8652132525958273, 0.8767087528273612, 0.882139315035025]

# Exercise

This exerise uses the ``WOLF_MODEL`` and the data ``WOLF_DF``.
Only fit the parameters ``J1_k1``, ``J1_ki``, and ``J1_n``.

1. Do 2, 10, 20, and 100 fold cross validations. Save the fold qualities for each.
How does the mean value of quality change with the number of folds?

1. Compute the standard deviations of the quality scores.
Can you explain the trend in standard deviations?

1. Plot the distribution of quality scores for 20 and 100 folds.
What can you say about these distributions?


## (1) Calculate Cross Validations

In [243]:
model = te.loada(WOLF_MODEL)
colnames = list(WOLF_DF.columns)
observedData = NamedTimeseries(dataframe=WOLF_DF)

In [244]:
upper = 1e5
parametersToFit = [
      SBstoat.Parameter("J1_k1", lower=0, value=1, upper=upper), #550
      SBstoat.Parameter("J1_Ki", lower=0, value=1, upper=upper), #1
      SBstoat.Parameter("J1_n", lower=0, value=1, upper=upper), #4                                                                                                                                                                 
]

In [247]:
result = {}
for numFold in [2, 10, 20, 100]:
    qualities = crossValidate(model, observedData, parametersToFit,
                          colnames, numFold,
                          fitterMethods=["differential_evolution"],
                          endTime=5)
    result[numFold] = qualities


23375.882562:     (Roadrunner exception: : CVODE Error: CV_CONV_FAILURE: Convergence test failures occurred too many times (= MXNCF = 10) during one internal timestep or occurred with |h| = hmin.; In virtual double rr::CVODEIntegrator::integrate(double, double))

23375.892942:     (Roadrunner exception: : CVODE Error: CV_CONV_FAILURE: Convergence test failures occurred too many times (= MXNCF = 10) during one internal timestep or occurred with |h| = hmin.; In virtual double rr::CVODEIntegrator::integrate(double, double))

23375.902675:     (Roadrunner exception: : CVODE Error: CV_CONV_FAILURE: Convergence test failures occurred too many times (= MXNCF = 10) during one internal timestep or occurred with |h| = hmin.; In virtual double rr::CVODEIntegrator::integrate(double, double))

23375.912643:     (Roadrunner exception: : CVODE Error: CV_CONV_FAILURE: Convergence test failures occurred too many times (= MXNCF = 10) during one internal timestep or occurred with |h| = hmin.; In virtual

In [248]:
result

{2: [0.8711255474151747, 0.8591564031826501],
 10: [0.8666633543322432,
  0.860143861636022,
  0.8628823369682499,
  0.8508738496770591,
  0.8811344640139932,
  0.8542620896742175,
  0.8740737210757702,
  0.8809320053812665,
  0.8659802843937492,
  0.8485653840861126],
 20: [0.8665220310620622,
  0.8387638195495389,
  0.8513322537443666,
  0.8081768230584291,
  0.861612555261823,
  0.7708922149690439,
  0.8360751118238482,
  0.8317991689198504,
  0.8378999743653367,
  0.837519413092948,
  0.8686406553936153,
  0.8740542516590821,
  0.8719658146973213,
  0.8773683708359532,
  0.896286145294775,
  0.9109678838204935,
  0.9069052273700092,
  0.9207151287857237,
  0.8903019901218217,
  0.8603121553138005],
 100: [0.8416405735239224,
  0.7823277639053389,
  0.855395441415933,
  0.7177691940482154,
  0.8118669459107951,
  0.6546509561843278,
  0.8559587511383374,
  0.7369607768982854,
  0.8409629109151353,
  0.8789937540978635,
  0.8693314928259588,
  0.8674172251238691,
  0.8722659483222801